In [23]:
import numpy as np
import torch
import os
import sympy as sp

# Device 
print("Versión de PyTorch:", torch.__version__)
print("CUDA disponible?:", torch.cuda.is_available())
print("MPS disponible?:", torch.backends.mps.is_available())

if torch.backends.mps.is_available():
    device = torch.device("mps")        # GPU Apple Silicon
    print("Usando GPU MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda")       # Para PCs con NVIDIA
    print("Usando GPU CUDA")
else:
    device = torch.device("cpu")
    print("Usando CPU")


Versión de PyTorch: 2.1.0
CUDA disponible?: False
MPS disponible?: True
Usando GPU MPS


# Ecuación de Poisson

In [3]:
# Definiendo una semilla para reproducibilidad
def set_seed(seed = 88):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    torch.use_deterministic_algorithms(True, warn_only=True)

set_seed()

Considere el siguiente problema: Hallar $u=u(x,y)$, satisfaciendo:

$$\dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial y^2} + 2\pi^2 \sin(\pi x)\sin(\pi y) = 0$$
sobre $\Omega = (1,0)\times(1,0)$, y con la condición $u=0$ sobre todo el borde de $\Omega$. 

La solucion del problema anterior viene dada por:
$$u(x,y) = \sin(\pi x)\sin(\pi y).$$ 

In [4]:
def f_np(x,y):
    '''Función fuente para la ecuación de Poisson.'''
    return 2*np.pi**2 * np.sin(np.pi*x)*np.sin(np.pi*y)

def f_th(x,y):
    '''Función fuente para la ecuación de Poisson.'''
    return 2*torch.pi**2 * torch.sin(torch.pi*x)*torch.sin(torch.pi*y)

In [5]:
# Verificación de la solución analítica
x, y = sp.symbols('x, y')

f_sp = 2*sp.pi**2 * sp.sin(sp.pi*x)*sp.sin(sp.pi*y)
u_sp = sp.sin(sp.pi*x)*sp.sin(sp.pi*y)

Poisson_eq = sp.diff(u_sp, x, 2) + sp.diff(u_sp, y, 2) + f_sp
print(Poisson_eq.simplify())

del x, y 

0


In [6]:
# Solución del problema anterior
def u_exact(x,y):
    '''Solución exacta de la ecuación de Poisson.'''
    return np.sin(np.pi*x)*np.sin(np.pi*y)

## A Plain PINN Class

In [38]:
class PINN2D(torch.nn.Module):
    def __init__(self, Nn, NL, actit = torch.tanh):
        super(PINN2D, self).__init__()
        self.actit = actit
        self.hidden = torch.nn.ModuleList([torch.nn.Linear(2, Nn)])
        self.hidden.extend([torch.nn.Linear(Nn,Nn) for _ in range(NL-1)])
        self.out = torch.nn.Linear(Nn,1)

        self.to(device)
    
    def forward(self, x):
        z = x
        for layer in self.hidden:
            z = self.actit(layer(z))
        return self.out(z)
        

In [45]:
def count_parameters(pinn):
    ''' Compute the total number of trainable parameters for an NN pinn'''
    return sum(p.numel() for p in pinn.parameters() if p.requires_grad)

def print_model_params(pinn, print_table = True):
    P_total = 0
    P_star = 0

    if print_table:
        print(f"{'Layer':<20}{'Param Shape':<15}{'# params':<15}"
              f"{'affine_dim':<15}")
        print("-" * 61)
    
    for name, param in pinn.named_parameters():
        if param.requires_grad:
            affin_dim = 0
            if param.dim() == 2:
                affin_dim = param.size(1)
            elif param.dim() == 1:
                affin_dim = 1

            P_star += affin_dim
            num_params = param.numel()
            P_total += num_params

            if print_table:
                print(f'{name:<20}{str(list(param.shape)):<15}'
                      f'{num_params:<15}{affin_dim}')
    
    if print_table:
        print("-" * 61)
    
    print(f'P*(Total pseudo-dimension of the affine space) = {P_star}')
    print(f'Total trainable parameters: {P_total}')

    return P_star, P_total

In [48]:
p = 2
Nn = 50
NL = 2
P_star = (p+1) + (Nn +1) * NL

print(f"P* (Total pseudo-dimension of affine space) = {P_star}")


P* (Total pseudo-dimension of affine space) = 105


In [49]:
pinn = PINN2D(Nn, NL)
P_star, P_total = print_model_params(pinn)

Layer               Param Shape    # params       affine_dim     
-------------------------------------------------------------
hidden.0.weight     [50, 2]        100            2
hidden.0.bias       [50]           50             1
hidden.1.weight     [50, 50]       2500           50
hidden.1.bias       [50]           50             1
out.weight          [1, 50]        50             50
out.bias            [1]            1              1
-------------------------------------------------------------
P*(Total pseudo-dimension of the affine space) = 105
Total trainable parameters: 2751


In [43]:
count_parameters(pinn)

111

In [40]:
pinn.hidden

ModuleList(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1-3): 3 x Linear(in_features=5, out_features=5, bias=True)
)

In [35]:
# for q in pinn.parameters():
#     print(q)